#  Тестовое задание для стражировки в Avito  
## Автор: Павленко Арсений 
## Часть 5. Обучение классификатора 

In [1]:
import numpy as np
import pandas as pd

In [2]:
from os.path import join
import pickle

In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

## Загрузка данных

In [4]:
PATH_TO_TRAIN_VECTORS = '../02_Files/Fasttext/Vectors/train/'
PATH_TO_VALID_VECTORS = '../02_Files/Fasttext/Vectors/valid/'
PATH_TO_TEST_VECTORS = '../02_Files/Fasttext/Vectors/test/'

In [5]:
PATH_TO_TRAIN_TARGET = '../02_Files/Train_test_split/train/'
PATH_TO_VALID_TARGET = '../02_Files/Train_test_split/valid/'

In [6]:
PATH_TO_TRAIN_PRICE = '../02_Files/Train_test_split/train/'
PATH_TO_VALID_PRICE = '../02_Files/Train_test_split/valid/'
PATH_TO_TEST_PRICE = '../02_Files/Train_test_split/test/'

Реализуем функцию `to_array` для преобразовния векторов, полученных из fasttext'а в numpy array.

In [7]:
def to_array(file_name):
    list_ = []
    with open(file_name) as f_in:
        for line in f_in:
            vector = [float(value) for value in line.strip().split()]
            list_.append(vector)
    return  np.array(list_)

In [8]:
features = ['title', 'description']

In [9]:
train = []
valid = []
test = []
for name in features:
    train_vec = to_array(join(PATH_TO_TRAIN_VECTORS, name + '.txt'))
    valid_vec = to_array(join(PATH_TO_VALID_VECTORS, name + '.txt'))
    test_vec = to_array(join(PATH_TO_TEST_VECTORS, name + '.txt'))
    train.append(train_vec)
    valid.append(valid_vec)
    test.append(test_vec)   

Загрузим также данные соответствующие признаку `price`

In [10]:
train_price = pd.read_csv(join(PATH_TO_TRAIN_PRICE, 'price.csv'), index_col=0).values
valid_price = pd.read_csv(join(PATH_TO_VALID_PRICE, 'price.csv'), index_col=0).values
test_price = pd.read_csv(join(PATH_TO_TEST_PRICE, 'price.csv'), index_col=0).values

Объеденим полученные вектора для признаков в один вектор

In [11]:
train_vec = np.concatenate([train[0],train[1], train_price], axis=1)
valid_vec = np.concatenate([valid[0],valid[1], valid_price], axis=1)
test_vec = np.concatenate([test[0],test[1], test_price], axis=1)

In [12]:
y_train = pd.read_csv(join(PATH_TO_TRAIN_TARGET, 'category_id.csv'), index_col=0,)
y_valid = pd.read_csv(join(PATH_TO_VALID_TARGET, 'category_id.csv'), index_col=0)

Объеденим полученные вектора DataFrame'ы, соответственно обучающий, валидационный и тестовый.

In [13]:
test_index = pd.read_csv(join(PATH_TO_TEST_PRICE, 'price.csv'), index_col=0).index # индексы для тестовой выборки

In [14]:
X_train = pd.DataFrame(train_vec, index=y_train.index) 
X_valid = pd.DataFrame(valid_vec, index=y_valid.index)
X_test = pd.DataFrame(test_vec, index=test_index)

In [15]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
item_id,,,,,,,,,,,,,,,,,,,,,
172692,0.005742,0.030685,0.070216,0.015761,0.024770,0.010076,0.068790,-0.050843,0.024449,-0.046310,...,0.005306,0.013800,0.020399,0.000901,-0.005830,-0.014996,0.014443,0.018459,0.011445,95000.0
257996,-0.126580,0.062789,0.119590,-0.075800,-0.012804,-0.053273,-0.108910,-0.053839,-0.016661,0.060833,...,-0.042685,-0.004235,0.001758,-0.010188,0.059751,0.020915,0.035973,0.051084,0.028380,1500.0
164493,-0.095908,-0.046235,-0.132620,-0.039315,0.006276,-0.002027,-0.115610,0.035406,-0.082879,-0.036019,...,0.000955,-0.035981,0.064088,-0.090798,-0.039223,0.043763,0.019959,-0.077008,-0.029363,500.0
473151,0.039320,-0.019199,-0.013060,-0.031113,-0.027573,-0.029855,0.033939,0.010369,-0.051529,-0.004660,...,0.042684,0.078721,0.030330,0.006828,0.051368,-0.002777,-0.033264,-0.010058,0.011107,880.0
389754,0.076765,-0.024362,-0.000550,0.028319,-0.036244,-0.022349,-0.071497,-0.008150,0.054179,0.004787,...,0.035710,0.012145,0.013659,0.029907,-0.003645,0.019328,-0.042271,-0.048457,-0.011045,20200.0


In [16]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
item_id,,,,,,,,,,,,,,,,,,,,,
489517,0.032086,-0.031176,-0.031888,0.038184,0.037083,0.007567,-0.049036,-0.001727,0.073209,0.014471,...,0.033818,0.048582,-0.057846,-0.002387,0.009992,-0.011164,-0.070159,-0.025010,-0.009610,10000.0
489518,0.115080,-0.052715,-0.033380,0.039130,-0.140730,0.107640,-0.102830,0.083248,-0.016148,0.133850,...,0.011301,-0.018572,-0.043967,-0.014199,0.030316,-0.000687,0.022450,0.009937,0.031191,12500.0
489519,0.027839,0.031515,0.028885,-0.041949,0.015269,-0.007168,0.039929,0.037295,-0.003060,-0.058486,...,0.005325,0.014602,0.027899,-0.003676,-0.003463,-0.014884,-0.043661,-0.031292,-0.004782,250.0
489520,0.002531,0.034186,0.069321,0.046804,0.039896,-0.055307,0.024447,-0.092671,0.063209,-0.066705,...,0.030023,0.026260,0.004017,-0.004753,-0.016452,-0.037610,-0.001836,0.025215,0.015790,1700.0
489521,0.034305,-0.042247,0.008953,-0.067982,-0.069849,-0.020763,0.046388,0.023261,0.015052,-0.078742,...,-0.019857,0.033304,-0.035200,-0.014989,0.040558,0.010288,-0.022660,-0.008668,0.000887,1000.0


In [17]:
y_train.head()

,category_id
item_id,
172692,33
257996,10
164493,8
473151,53
389754,25


# Обучение  xgboost

Реализуем функцию `write_to_submission_file` для сохранения полученных предсказаний.

In [18]:
def write_to_submission_file(predicted_labels, out_file,
                             target='category_id', index_label="item_id"):
    predicted_df = pd.DataFrame(predicted_labels.astype(int),
                                index = test_index,
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [19]:
import xgboost as xgb

In [20]:
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)
dtest = xgb.DMatrix(X_test)

Обучим Xgboost со следующим набором параметров:

*(по хорошему надо их подбирать на кроссвалидации но из-за нехватки времени на данный момент я использую интуитивно подобранные параметры. В скором времени исправлю)*

In [21]:
params = {
    'objective':'multi:softmax',
    'num_class': 54,
    'max_depth':3,
    'subsample':0.618,
    'colsample_bytree':0.618,
    'silent':1,
    'eta':0.2,
    'eval_metric': 'merror',
}

num_rounds = 100

Посмотрим на ошибки в процессе обучения

In [22]:
watchlist  = [(dvalid,'valid'), (dtrain,'train')]

In [23]:
%%time
xgb_model_100 = xgb.train(params, dtrain, num_rounds, watchlist)

[0]	valid-merror:0.354564	train-merror:0.325488
[1]	valid-merror:0.307846	train-merror:0.274239
[2]	valid-merror:0.271033	train-merror:0.234367
[3]	valid-merror:0.247556	train-merror:0.20904
[4]	valid-merror:0.230508	train-merror:0.19038
[5]	valid-merror:0.21786	train-merror:0.176804
[6]	valid-merror:0.207817	train-merror:0.165731
[7]	valid-merror:0.199512	train-merror:0.156753
[8]	valid-merror:0.191849	train-merror:0.148892
[9]	valid-merror:0.186357	train-merror:0.142377
[10]	valid-merror:0.181004	train-merror:0.136698
[11]	valid-merror:0.177009	train-merror:0.132146
[12]	valid-merror:0.173159	train-merror:0.12808
[13]	valid-merror:0.169795	train-merror:0.124689
[14]	valid-merror:0.166683	train-merror:0.121416
[15]	valid-merror:0.164217	train-merror:0.118408
[16]	valid-merror:0.161725	train-merror:0.116018
[17]	valid-merror:0.159522	train-merror:0.113361
[18]	valid-merror:0.15756	train-merror:0.11145
[19]	valid-merror:0.155902	train-merror:0.109685
[20]	valid-merror:0.154158	train-mer

In [24]:
pred_100 = xgb_model_100.predict(dvalid)

Посмотрим на получившуюся точность на отложенной выборке и сохраним предсказания для тестовых данных

In [26]:
print()
print("Accuracy: {}".format(round(accuracy_score(y_valid, pred_100 ), 3)))
print()


test_pred_100 = xgb_model_100.predict(dtest)

write_to_submission_file(test_pred_100, 'y_test_100.csv')

()
Accuracy: 0.876
()
